In [1]:
import catboost
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from mclLib.featurenames import *
from mclLib.cleaner import *
from mclLib.server_info import *

In [2]:
engine = create_engine(mysql_server_uri)
conn = engine.connect()

In [3]:
''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [4]:

''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) 
br = br_full_data[get_filter]

train_test_day_split = '2023-03-01'
train_test_day_filter = (br['dTradeTime'] < train_test_day_split)

''' get features name'''
feature_names_102 =  f_name_102
feature_size = len(feature_names_102)



In [5]:

''' set Train X y data '''
br_train = br[train_test_day_filter]
X_train = br_train[feature_names_102].to_numpy(dtype=np.float64)
y_train_condition = (br_train['fMaxPowerAfterBuyWhile10'] >= 0.04)
y_train = np.where(y_train_condition, 1, 0)


''' set Test X y data '''
br_test = br[~train_test_day_filter]
X_test = br_test[feature_names_102].to_numpy(dtype=np.float64)
y_test_condition = (br_test['fMaxPowerAfterBuyWhile10'] >= 0.04)
y_test = np.where(y_test_condition, 1, 0)


print('X_train : ', X_train.shape)
print('y_train : ', y_train.shape)
print('X_test  : ', X_test.shape)
print('y_test  : ', y_test.shape)

X_train :  (0, 102)
y_train :  (0,)
X_test  :  (14265, 102)
y_test  :  (14265,)


In [7]:
model = catboost.CatBoostClassifier(loss_function='Logloss')

In [8]:
model.fit(X_train, y_train)

Learning rate set to 0.074444
0:	learn: 0.6675205	total: 321ms	remaining: 5m 20s
1:	learn: 0.6462080	total: 578ms	remaining: 4m 48s
2:	learn: 0.6284134	total: 872ms	remaining: 4m 49s
3:	learn: 0.6126194	total: 1.14s	remaining: 4m 43s
4:	learn: 0.5996870	total: 1.32s	remaining: 4m 23s
5:	learn: 0.5885435	total: 1.52s	remaining: 4m 12s
6:	learn: 0.5799311	total: 1.7s	remaining: 4m
7:	learn: 0.5719589	total: 1.86s	remaining: 3m 50s
8:	learn: 0.5650657	total: 2.06s	remaining: 3m 47s
9:	learn: 0.5591374	total: 2.27s	remaining: 3m 45s
10:	learn: 0.5539037	total: 2.41s	remaining: 3m 36s
11:	learn: 0.5493843	total: 2.55s	remaining: 3m 30s
12:	learn: 0.5457741	total: 2.85s	remaining: 3m 36s
13:	learn: 0.5421701	total: 3.09s	remaining: 3m 37s
14:	learn: 0.5394282	total: 3.24s	remaining: 3m 33s
15:	learn: 0.5372254	total: 3.4s	remaining: 3m 29s
16:	learn: 0.5348419	total: 3.51s	remaining: 3m 23s
17:	learn: 0.5330082	total: 3.63s	remaining: 3m 18s
18:	learn: 0.5311082	total: 3.74s	remaining: 3m 13

In [9]:
# Save model to ONNX-ML format
model.save_model(
    "./onnx/catboost_test.onnx",
    format="onnx",
    export_parameters={
        'onnx_domain': 'ai.catboost',
        'onnx_model_version': 1,
        'onnx_doc_string': 'test model for BinaryClassification',
        'onnx_graph_name': 'CatBoostModel_for_BinaryClassification'
    }
)

In [ ]:
cleanAll()